In [1]:
import numpy as np
import pandas as pd

from functools import total_ordering 

# Rock paper scissors tournament

In [86]:
@total_ordering
class rps:
    move = ""

    def __init__(self, move):
        self.move = move

    def _is_valid_operand(self, other):
        return (other.move in set(["rock", "paper", "scissors"]))

    def __eq__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return (str(self.move) == str(other.move))
        
    def __gt__(self, other):
        if self.move == "rock" and other.move == "scissors":
            return True
        elif self.move == "paper" and other.move == "rock":
            return True
        elif self.move == "scissors" and other.move == "paper":
            return True
        else:
            return False

    def __lt__(self, other):
        return other.__gt__(self)

    def __hash__(self) -> int:
        if self.move == "rock":
            return 0
        elif self.move == "paper":
            return 1
        elif self.move == "scissors":
            return 2

    def win(self):
        if self.move == "rock":
            return "paper"
        elif self.move == "paper":
            return "scissors"
        elif self.move == "scissors":
            return "rock"

    def lose(self):
        if self.move == "rock":
            return "scissors"
        elif self.move == "paper":
            return "rock"
        elif self.move == "scissors":
            return "paper"

In [3]:
rock = rps("rock")
paper = rps("paper")
scissors = rps("scissors")

In [4]:
assert(rock == rock)
assert(paper == paper)
assert(scissors == scissors)

assert(rock < paper)
assert(paper > rock)
assert(rock > scissors)
assert(scissors < rock)
assert(paper < scissors)
assert(scissors > paper)

assert(not scissors < scissors)
assert(not scissors > scissors)
assert(not paper < paper)
assert(not paper > paper)
assert(not rock < rock)
assert(not rock > rock)

In [15]:
score_table = {
    "rock": 1,
    "paper": 2,
    "scissors": 3,
    "loss": 0,
    "draw": 3,
    "win": 6
}

code = {
    "X": "rock",
    "Y": "paper",
    "Z": "scissors",
    "A": "rock",
    "B": "paper",
    "C": "scissors",
}

In [6]:
def play(opponent, me, score=score_table):
    my_points = 0
    opponent_points = 0
    if opponent > me:
        my_points += score["loss"]
        opponent_points += score["win"]
    elif me > opponent:
        my_points += score["win"]
        opponent_points += score["loss"]
    else:
        my_points += score["draw"]
        opponent_points += score["draw"]
    my_points += score[me.move]
    opponent_points += score[opponent.move]

    return my_points, opponent_points

In [7]:
game = """A Y
B X
C Z"""

In [19]:
def play_game(game):
    record = []
    for round in game.split("\n"):
        opp_code, my_code = round.split()
        opponent = rps(code[opp_code])
        me = rps(code[my_code])
        my_score, their_score = play(opponent, me)
        record.append([opp_code, my_code, their_score, my_score])
    record = pd.DataFrame(record, columns=["opp_move", "my_move", "opp_score", "my_score"])
    return record

In [20]:
mock = play_game(game)

In [21]:
mock["my_score"].sum()

15

In [22]:
with open("../data/day02", "r") as infile:
    data = infile.read().rstrip()

In [23]:
real = play_game(data)

In [24]:
real["my_score"].sum()

15523

In [25]:
new_code = {
    "X": "loss",
    "Y": "paper",
    "Z": "win",
    "A": "rock",
    "B": "paper",
    "C": "scissors",
}

In [50]:
def update_moves(game, code):
    record = []
    for round in game.split("\n"):
        opp_code, my_code = round.split()
        opponent = rps(code[opp_code])
        if my_code == "X":
            me = rps(opponent.lose())
        elif my_code == "Z":
            me = rps(opponent.win())
        elif my_code == "Y":
            me = opponent
        my_score, their_score = play(opponent, me)
        record.append([opponent.move, me.move, their_score, my_score, my_code])
    record = pd.DataFrame(record, columns=["opp_move", "my_move", "opp_score", "my_score", "directive"])
    return record

In [87]:
update_real = update_moves(data, code)

In [90]:
update_real.loc[11]

opp_move     scissors
my_move         paper
opp_score           9
my_score            2
directive           X
Name: 11, dtype: object

In [91]:
update_real["my_score"].sum()

15702